## Formula 1 Dashboard

Tilda Davies
DS 4003
--------

## Data Description

The data was found from Kaggle. It was in the form of multiple CSVs that are all about different aspects of Formula 1 racing. In cleaning the data, I plan on combining these datasets into one larger dataset so that I can build my dashboard from one singular dataframe as opposed to multiple. The provenance of the data is found [here](http://ergast.com/mrd/). 

## Data Cleaning

In [43]:
# import dependencies
import pandas as pd
import seaborn as sns
import plotly.express as px
import datetime
from datetime import time

In [44]:
# read in datasets
drivers = pd.read_csv("/Users/tilliedavies/Desktop/DS 4003/F1 Data/drivers.csv")
lap_times = pd.read_csv("/Users/tilliedavies/Desktop/DS 4003/F1 Data/lap_times.csv")
pit_stops = pd.read_csv("/Users/tilliedavies/Desktop/DS 4003/F1 Data/pit_stops.csv")
races = pd.read_csv("/Users/tilliedavies/Desktop/DS 4003/F1 Data/races.csv")
results = pd.read_csv("/Users/tilliedavies/Desktop/DS 4003/F1 Data/results.csv")

In [45]:
# clean each individual csv (remove unneccesary columns)
races = races.drop(columns=['time','url','fp1_date','fp1_time','fp2_date','fp2_time','fp3_date','fp3_time','quali_date','quali_time','sprint_date','sprint_time','circuitId','date'])
drivers = drivers.drop(columns=['number','url','code','dob','nationality'])
pit_stops = pit_stops.drop(columns=['time','duration'])
lap_times = lap_times.drop(columns=['time'])
results = results.drop(columns=['constructorId','position','positionOrder', 'statusId','resultId','number','time'])

In [46]:
# merge all csvs into one big dataset
races1 = pd.merge(lap_times,pit_stops, on=['raceId', 'driverId', 'lap'], how='outer')
races2 = pd.merge(races1, races, on='raceId', how='outer')
races3 = pd.merge(races2, drivers, on='driverId', how='outer')
F1 = pd.merge(races3, results, on=['driverId','raceId'], how='outer')

In [47]:
# rename columns
F1.rename(columns={'stop':'pitStop','milliseconds_y':'pitStopDuration','milliseconds_x':'lapTime'}, inplace=True)

In [48]:
# check null counts and drop rows that have nan's in the raceId and year column
F1.isnull( ).sum()
F1.dropna(subset=['raceId'],inplace=True)
F1.dropna(subset=['year'],inplace=True)
F1.dropna(subset=['driverId'],inplace=True)

In [49]:
# Get rid of rows that contain '\N' in these columns, irrelevant to our analysis.
F1 = F1[F1['rank'] != '\\N']
F1 = F1[F1['fastestLapSpeed'] != '\\N']
F1 = F1[F1['milliseconds'] != '\\N']

### Fix data types

In [50]:
# convert object columns to numeric where they should be
F1['milliseconds'] = pd.to_numeric(F1['milliseconds'])
F1['fastestLapSpeed'] = pd.to_numeric(F1['fastestLapSpeed'])

In [51]:
F1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 228379 entries, 0 to 551741
Data columns (total 22 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   raceId           228379 non-null  float64
 1   driverId         228379 non-null  float64
 2   lap              228379 non-null  float64
 3   position         228379 non-null  float64
 4   lapTime          228379 non-null  float64
 5   pitStop          5500 non-null    float64
 6   pitStopDuration  5500 non-null    float64
 7   year             228379 non-null  float64
 8   round            228379 non-null  float64
 9   name             228379 non-null  object 
 10  driverRef        228379 non-null  object 
 11  forename         228379 non-null  object 
 12  surname          228379 non-null  object 
 13  grid             228379 non-null  float64
 14  positionText     228379 non-null  object 
 15  points           228379 non-null  float64
 16  laps             228379 non-null  float64
 

In [52]:
F1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 228379 entries, 0 to 551741
Data columns (total 22 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   raceId           228379 non-null  float64
 1   driverId         228379 non-null  float64
 2   lap              228379 non-null  float64
 3   position         228379 non-null  float64
 4   lapTime          228379 non-null  float64
 5   pitStop          5500 non-null    float64
 6   pitStopDuration  5500 non-null    float64
 7   year             228379 non-null  float64
 8   round            228379 non-null  float64
 9   name             228379 non-null  object 
 10  driverRef        228379 non-null  object 
 11  forename         228379 non-null  object 
 12  surname          228379 non-null  object 
 13  grid             228379 non-null  float64
 14  positionText     228379 non-null  object 
 15  points           228379 non-null  float64
 16  laps             228379 non-null  float64
 

# EDA

In [53]:
# overview of non-null counts and data types
F1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 228379 entries, 0 to 551741
Data columns (total 22 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   raceId           228379 non-null  float64
 1   driverId         228379 non-null  float64
 2   lap              228379 non-null  float64
 3   position         228379 non-null  float64
 4   lapTime          228379 non-null  float64
 5   pitStop          5500 non-null    float64
 6   pitStopDuration  5500 non-null    float64
 7   year             228379 non-null  float64
 8   round            228379 non-null  float64
 9   name             228379 non-null  object 
 10  driverRef        228379 non-null  object 
 11  forename         228379 non-null  object 
 12  surname          228379 non-null  object 
 13  grid             228379 non-null  float64
 14  positionText     228379 non-null  object 
 15  points           228379 non-null  float64
 16  laps             228379 non-null  float64
 

## How many observations does the data contain?

In [54]:
# use shape function to get the (rows, columns)
F1.shape

(228379, 22)

Data has 228379 observations.

## How many unique categories are there for categorical variables?

In [55]:
F1.nunique()

raceId               377
driverId              86
lap                   87
position              24
lapTime            62367
pitStop                6
pitStopDuration     4404
year                  20
round                 22
name                  40
driverRef             86
forename              78
surname               84
grid                  25
positionText          20
points                28
laps                  33
milliseconds        3841
fastestLap            78
rank                  22
fastestLapTime      3686
fastestLapSpeed     3717
dtype: int64

This helps to be able to identify which variables to use as unique identifiers, for example there are 86 driverId's or driverRef's but only 84 surnames. This also ensures the data is accurate because there are 377 unique raceId's as well as 377 unique race dates. 

## How much missing data is there per variable?

In [56]:
F1.isnull().sum()

raceId                  0
driverId                0
lap                     0
position                0
lapTime                 0
pitStop            222879
pitStopDuration    222879
year                    0
round                   0
name                    0
driverRef               0
forename                0
surname                 0
grid                    0
positionText            0
points                  0
laps                    0
milliseconds            0
fastestLap              0
rank                    0
fastestLapTime          0
fastestLapSpeed         0
dtype: int64

This is good! Pit stops typically don't occur during every lap of a race so it is expected to have nulls for the variables about pit stops. 

## What are the distributions of continuous variables? Are there outliers?

In [57]:
# first need to subset the F1 df to only the continuous variables
cont_vars = F1[['lapTime','pitStopDuration','milliseconds','fastestLapSpeed']]

# use dsecribe function to get numerical summaries of each variable
cont_vars.describe()

,lapTime,pitStopDuration,milliseconds,fastestLapSpeed
count,2.283790e+05,5.500000e+03,2.283790e+05,228379.000000
mean,9.848223e+04,9.578994e+04,5.910809e+06,205.219991
std,8.939874e+04,3.326788e+05,1.073867e+06,21.177932
min,5.540400e+04,1.317300e+04,4.252092e+06,148.589000
25%,8.267450e+04,2.170500e+04,5.359668e+06,195.201000
50%,9.387200e+04,2.339000e+04,5.675199e+06,206.346000
75%,1.041980e+05,2.583350e+04,6.052930e+06,218.566000
max,7.506656e+06,3.069017e+06,1.474314e+07,257.320000


Appears to be an outlier in lapTime shown as the max. 

## Data Dictionary

| Variable Name | Definition |
|-----------------|----------------------------------|
| raceID   | Unique identifier for each Formula 1 race.   |
| driverId   | Unique identifier for each Formula 1 driver.   |
| lap    | Lap number.  |
| position | What position the driver is in during each lap of each race.|
| lapTime | Duration of each lap (in milliseconds)|
|pitStop|Number of pit stop, 1 being the first pit stop taken.|
|pitStopDuration|Duration of pit stop in milliseconds.|
|round | Round number of Grand Prix. |
|name| Name of race| 
| date| Date of race|
|driverRef|Unique identifier for each driver that is typically last name, with a first initial when necessary|
| points| Points scored in each race.|
| laps| Total amount of laps in that race (not consistent across all Grand Prixes)|
| milliseconds| Total time to complete race (in milliseconds)|
| fastestLap| Lap in which it took the least amount of time to complete.|
| fastestLapSpeed| Recorded speed of the fastestLap (in mph)|

## UI Components
- Slider to select year or range of years.
- Dropdown to select specific Grand Prixes.
- Search bar or other quick way for user to find their favorite driver quickly.
- Select two drivers via dropdown to compare performance. 

## Data Visualization Brainstorm
- Line graph showing points accumulation over one season, colored by driver.
- Bar chart showing wins over time.
- Line graph showing speed by lap.
- Scatter plot of pit stop duration vs final rank in race, this could be filtered on Grand Priz using a multi-select dropdown. 

In [58]:
F1=F1[F1['year']>=2009]

In [59]:
F1.to_csv('data.csv',index=False)